<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-16.-Coroutines" data-toc-modified-id="Chapter-16.-Coroutines-1">Chapter 16. Coroutines</a></span><ul class="toc-item"><li><span><a href="#simple-coroutine" data-toc-modified-id="simple-coroutine-1.1">simple coroutine</a></span></li><li><span><a href="#Coroutine-states" data-toc-modified-id="Coroutine-states-1.2">Coroutine states</a></span></li><li><span><a href="#a-running-average-coroutine" data-toc-modified-id="a-running-average-coroutine-1.3">a running average coroutine</a></span><ul class="toc-item"><li><span><a href="#Using-@wraps-decorator" data-toc-modified-id="Using-@wraps-decorator-1.3.1">Using @wraps decorator</a></span></li></ul></li><li><span><a href="#Coroutine-Termination-and-Exception-Handling" data-toc-modified-id="Coroutine-Termination-and-Exception-Handling-1.4">Coroutine Termination and Exception Handling</a></span><ul class="toc-item"><li><span><a href="#generator.throw()" data-toc-modified-id="generator.throw()-1.4.1">generator.throw()</a></span></li><li><span><a href="#generator.close()" data-toc-modified-id="generator.close()-1.4.2">generator.close()</a></span></li></ul></li><li><span><a href="#returning-values-from-coroutines" data-toc-modified-id="returning-values-from-coroutines-1.5">returning values from coroutines</a></span><ul class="toc-item"><li><span><a href="#returning-values-from-coroutines-without-using-yield-from" data-toc-modified-id="returning-values-from-coroutines-without-using-yield-from-1.5.1">returning values from coroutines without using <code>yield from</code></a></span></li><li><span><a href="#usingyield-from" data-toc-modified-id="usingyield-from-1.5.2">using<code>yield from</code></a></span></li></ul></li><li><span><a href="#Taxi-simulator" data-toc-modified-id="Taxi-simulator-1.6">Taxi simulator</a></span></li></ul></li></ul></div>

# Chapter 16. Coroutines
A coroutine is syntactically like a generator: just a function with the 'yield' keyword in its body. However, in a coroutine, 'yield' usually appears on the right side of an expression (e.g., 'datum = yield'), and it may or may not produce a value. 

If there is no expression after the 'yield' keyword, the generator yields 'None'. 

The coroutine may receive data from the caller, which uses '.send(datum)' instead of 'next(…)' to feed the coroutine. Usually, the caller pushes values into the coroutine.

coroutines can be used to implement cooperative multitasking: each coroutine yields control to a central scheduler so that other coroutines can be activated.

## simple coroutine

In [664]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7f8a03a1b270>

In [665]:
next(my_coro) # start coroutine to put in waiting at yield state
              # this line same as following:
              # my_coro.send(None)

-> coroutine started


In [666]:
my_coro.send(42) # send value to yield assignment 
                 # and resume to next yield, if any, 
                 # else raise StopIteration

-> coroutine received: 42


StopIteration: 

## Coroutine states

In [667]:
from inspect import getgeneratorstate

def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)
    
my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2) # Waiting to start execution.

'GEN_CREATED'

In [668]:
next(my_coro2)
getgeneratorstate(my_coro2) # Currently suspended at 1st yield expression.

-> Started: a = 14


'GEN_SUSPENDED'

In [669]:
my_coro2.send(28)
getgeneratorstate(my_coro2) # Currently suspended at 2nd yield expression.

-> Received: b = 28


'GEN_SUSPENDED'

In [670]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [671]:
getgeneratorstate(my_coro2) # Execution has completed.

'GEN_CLOSED'

## a running average coroutine

In [677]:
# Higher order function implementation of running averager
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total # make count, total free variables
        count += 1
        total += new_value
        return total / count

    return averager


In [678]:
f_avg = make_averager()
f_avg(10)

10.0

In [679]:
f_avg(30)

20.0

In [680]:
f_avg(5)

15.0

### Using @wraps decorator
Without the use of this decorator factory, the name of the example function would have been `wrapper`, and the docstring of the original example() would have been lost.

In [700]:
from functools import wraps

def my_decorator(f):
#     @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')
    
example()
print(example.__name__)
print(example.__doc__)

Calling decorated function
Called example function
example
Docstring


priming decorator primes `func` by advancing to first `yield`

In [711]:
from functools import wraps

def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`%%!
    
    incompatible with `yield from` coroutines. use the 
    asyncio.coroutine decorator instead"""
    @wraps(func)
    def primer(*args,**kwargs):   
        gen = func(*args,**kwargs)   
        next(gen)   
        return gen   
    return primer

In [722]:
# Coroutine implementation of running averager
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:   
        term = yield average   
        total += term
        count += 1
        average = total/count

In [723]:
coro_avg = averager()
coro_avg.send(10)

10.0

In [724]:
coro_avg.send(30)

20.0

In [725]:
coro_avg.send(5)

15.0

## Coroutine Termination and Exception Handling

In [726]:
class DemoException(Exception):
    """An exception type for the demonstration."""

@coroutine
def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:   
                print('*** DemoException handled. Continuing...')
            else:   
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

### generator.throw()

In [727]:
exc_coro = demo_exc_handling()
exc_coro.send(11)
exc_coro.send(22)
exc_coro.throw(DemoException)
getgeneratorstate(exc_coro)

-> coroutine started
-> coroutine received: 11
-> coroutine received: 22
*** DemoException handled. Continuing...


'GEN_SUSPENDED'

### generator.close()

In [728]:
exc_coro = demo_exc_handling()
exc_coro.send(11)
exc_coro.send(22)
exc_coro.close()
getgeneratorstate(exc_coro)

-> coroutine started
-> coroutine ending
-> coroutine received: 11
-> coroutine received: 22
-> coroutine ending


'GEN_CLOSED'

## returning values from coroutines

### returning values from coroutines without using `yield from`
the value of the return expression is smuggled to the caller as an attribute of the StopIteration exception.

In [729]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break   
        total += term
        count += 1
        average = total/count
    return Result(count, average)   

In [731]:
coro_avg = averager()
print(coro_avg.send(10))
print(coro_avg.send(30))
print(coro_avg.send(6.5))
print(coro_avg.send(None))

None
None
None


StopIteration: Result(count=3, average=15.5)

### using`yield from`
`yield from` is about allowing coroutines to return a result.

The main feature of `yield from` is to open a bidirectional channel from the outermost caller to the innermost subgenerator, so that values can be sent and yielded back and forth directly from them, and exceptions can be thrown all the way in without adding a lot of exception handling boilerplate code in the intermediate coroutines. 

In [732]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
        
list(gen())

['A', 'B', 1, 2]

can be written as 

In [733]:
def gen():
    yield from 'AB'
    yield from range(1,3)
        
list(gen())

['A', 'B', 1, 2]

In [734]:
# the subgenerator
# The generator obtained from the 
# <iterable> part of the yield from expression. 
def averager():   
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield 
        print(term)
        if term is None:   
            break
        total += term
        count += 1
        average = total/count
        
    print(Result(count, average))
    return Result(count, average)  # return value becomes 
                                   # result of yield from expression


# the delegating generator
# The generator function that contains the yield from 
# <iterable> expression.
def grouper(results, key):   
    while True:   
        results[key] = yield from averager()   


# the client code, a.k.a. the caller
def main(data):   
    results = {}
    for key, values in data.items():
        group = grouper(results, key)   
        next(group)   
        for value in values:
            group.send(value)   
        group.send(None)  # important! 

    print(results)  # uncomment to debug
    report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

40.9
38.5
44.3
42.2
45.2
41.7
44.5
38.0
40.6
44.5
None
Result(count=10, average=42.040000000000006)
1.6
1.51
1.4
1.3
1.41
1.39
1.33
1.46
1.45
1.43
None
Result(count=10, average=1.4279999999999997)
39.0
40.8
43.2
40.8
43.1
38.6
41.4
40.6
36.3
None
Result(count=9, average=40.422222222222224)
1.38
1.5
1.32
1.25
1.37
1.48
1.25
1.49
1.46
None
Result(count=9, average=1.3888888888888888)
{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [735]:
def taxi_process(ident, trips, start_time=0):   
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')   
    for i in range(trips):   
        time = yield Event(time, ident, 'pick up passenger')   
        time = yield Event(time, ident, 'drop off passenger')   

    yield Event(time, ident, 'going home')   
    # end of taxi process  

## Taxi simulator
The point of this example is to show a main loop processing events and driving coroutines by sending data to them. This is the basic idea behind asyncio

In [736]:
"""
Taxi simulator
==============

Driving a taxi from the console::

    >>> from taxi_sim import taxi_process
    >>> taxi = taxi_process(ident=13, trips=2, start_time=0)
    >>> next(taxi)
    Event(time=0, proc=13, action='leave garage')
    >>> taxi.send(_.time + 7)
    Event(time=7, proc=13, action='pick up passenger')
    >>> taxi.send(_.time + 23)
    Event(time=30, proc=13, action='drop off passenger')
    >>> taxi.send(_.time + 5)
    Event(time=35, proc=13, action='pick up passenger')
    >>> taxi.send(_.time + 48)
    Event(time=83, proc=13, action='drop off passenger')
    >>> taxi.send(_.time + 1)
    Event(time=84, proc=13, action='going home')
    >>> taxi.send(_.time + 10)
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
    StopIteration

Sample run with two cars, random seed 10. This is a valid doctest::

    >>> main(num_taxis=2, seed=10)
    taxi: 0  Event(time=0, proc=0, action='leave garage')
    taxi: 0  Event(time=5, proc=0, action='pick up passenger')
    taxi: 1     Event(time=5, proc=1, action='leave garage')
    taxi: 1     Event(time=10, proc=1, action='pick up passenger')
    taxi: 1     Event(time=15, proc=1, action='drop off passenger')
    taxi: 0  Event(time=17, proc=0, action='drop off passenger')
    taxi: 1     Event(time=24, proc=1, action='pick up passenger')
    taxi: 0  Event(time=26, proc=0, action='pick up passenger')
    taxi: 0  Event(time=30, proc=0, action='drop off passenger')
    taxi: 0  Event(time=34, proc=0, action='going home')
    taxi: 1     Event(time=46, proc=1, action='drop off passenger')
    taxi: 1     Event(time=48, proc=1, action='pick up passenger')
    taxi: 1     Event(time=110, proc=1, action='drop off passenger')
    taxi: 1     Event(time=139, proc=1, action='pick up passenger')
    taxi: 1     Event(time=140, proc=1, action='drop off passenger')
    taxi: 1     Event(time=150, proc=1, action='going home')
    *** end of events ***

See longer sample run at the end of this module.

"""

import random
import collections
import queue
import argparse
import time

DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

Event = collections.namedtuple('Event', 'time proc action')


# BEGIN TAXI_PROCESS
def taxi_process(ident, trips, start_time=0):
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')

    yield Event(time, ident, 'going home')
    # end of taxi process
# END TAXI_PROCESS


# BEGIN TAXI_SIMULATOR
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)

        # main loop of the simulation
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break

            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi:', proc_id, proc_id * '   ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR


def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action in ['leave garage', 'drop off passenger']:
        # new state is prowling
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # new state is trip
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1


def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS,
         seed=None):
    """Initialize random generator, build procs and run simulation"""
    if seed is not None:
        random.seed(seed)  # get reproducible results

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
             for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)


if __name__ == '__main__':

#     parser = argparse.ArgumentParser(
#                         description='Taxi fleet simulator.')
#     parser.add_argument('-e', '--end-time', type=int,
#                         default=DEFAULT_END_TIME,
#                         help='simulation end time; default = %s'
#                         % DEFAULT_END_TIME)
#     parser.add_argument('-t', '--taxis', type=int,
#                         default=DEFAULT_NUMBER_OF_TAXIS,
#                         help='number of taxis running; default = %s'
#                         % DEFAULT_NUMBER_OF_TAXIS)
#     parser.add_argument('-s', '--seed', type=int, default=3,
#                         help='random generator seed (for testing)')

#     args = parser.parse_args()
    args = namedtuple('args', 'end_time, taxis, seed')
    args.end_time, args.taxis, args.seed = DEFAULT_END_TIME, DEFAULT_NUMBER_OF_TAXIS, 3
    main(args.end_time, args.taxis, args.seed)


"""

Sample run from the command line, seed=3, maximum elapsed time=120::

# BEGIN TAXI_SAMPLE_RUN
$ python3 taxi_sim.py -s 3 -e 120
taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=8, proc=1, action='pick up passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 2        Event(time=15, proc=2, action='pick up passenger')
taxi: 2        Event(time=17, proc=2, action='drop off passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 2        Event(time=18, proc=2, action='pick up passenger')
taxi: 2        Event(time=25, proc=2, action='drop off passenger')
taxi: 1     Event(time=27, proc=1, action='drop off passenger')
taxi: 2        Event(time=27, proc=2, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='pick up passenger')
taxi: 2        Event(time=40, proc=2, action='drop off passenger')
taxi: 2        Event(time=44, proc=2, action='pick up passenger')
taxi: 1     Event(time=55, proc=1, action='pick up passenger')
taxi: 1     Event(time=59, proc=1, action='drop off passenger')
taxi: 0  Event(time=65, proc=0, action='drop off passenger')
taxi: 1     Event(time=65, proc=1, action='pick up passenger')
taxi: 2        Event(time=65, proc=2, action='drop off passenger')
taxi: 2        Event(time=72, proc=2, action='pick up passenger')
taxi: 0  Event(time=76, proc=0, action='going home')
taxi: 1     Event(time=80, proc=1, action='drop off passenger')
taxi: 1     Event(time=88, proc=1, action='pick up passenger')
taxi: 2        Event(time=95, proc=2, action='drop off passenger')
taxi: 2        Event(time=97, proc=2, action='pick up passenger')
taxi: 2        Event(time=98, proc=2, action='drop off passenger')
taxi: 1     Event(time=106, proc=1, action='drop off passenger')
taxi: 2        Event(time=109, proc=2, action='going home')
taxi: 1     Event(time=110, proc=1, action='going home')
*** end of events ***
# END TAXI_SAMPLE_RUN

"""

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=8, proc=1, action='pick up passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 2        Event(time=15, proc=2, action='pick up passenger')
taxi: 2        Event(time=17, proc=2, action='drop off passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 2        Event(time=18, proc=2, action='pick up passenger')
taxi: 2        Event(time=25, proc=2, action='drop off passenger')
taxi: 1     Event(time=27, proc=1, action='drop off passenger')
taxi: 2        Event(time=27, proc=2, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='pick up passenger')
taxi: 2        Event(time=40, proc=2, action='drop off passenger')
taxi: 2        Event(time=44, proc=2, action='pick up passenger')
taxi: 1     Event(time=55, proc=1, action='pick up passen

"\n\nSample run from the command line, seed=3, maximum elapsed time=120::\n\n# BEGIN TAXI_SAMPLE_RUN\n$ python3 taxi_sim.py -s 3 -e 120\ntaxi: 0  Event(time=0, proc=0, action='leave garage')\ntaxi: 0  Event(time=2, proc=0, action='pick up passenger')\ntaxi: 1     Event(time=5, proc=1, action='leave garage')\ntaxi: 1     Event(time=8, proc=1, action='pick up passenger')\ntaxi: 2        Event(time=10, proc=2, action='leave garage')\ntaxi: 2        Event(time=15, proc=2, action='pick up passenger')\ntaxi: 2        Event(time=17, proc=2, action='drop off passenger')\ntaxi: 0  Event(time=18, proc=0, action='drop off passenger')\ntaxi: 2        Event(time=18, proc=2, action='pick up passenger')\ntaxi: 2        Event(time=25, proc=2, action='drop off passenger')\ntaxi: 1     Event(time=27, proc=1, action='drop off passenger')\ntaxi: 2        Event(time=27, proc=2, action='pick up passenger')\ntaxi: 0  Event(time=28, proc=0, action='pick up passenger')\ntaxi: 2        Event(time=40, proc=2, ac

In [737]:
taxi = taxi_process(ident=13, trips=2, start_time=0)  
next(taxi)  
taxi.send(7)  

Event(time=7, proc=13, action='pick up passenger')

In [738]:
taxi.send(23)  

Event(time=23, proc=13, action='drop off passenger')

In [739]:
taxi.send(5)  

Event(time=5, proc=13, action='pick up passenger')

In [740]:
taxi.send(48)  

Event(time=48, proc=13, action='drop off passenger')

In [741]:
taxi.send(1)

Event(time=1, proc=13, action='going home')

In [742]:
taxi.send(10)  

StopIteration: 